In [3]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from unsloth import FastLanguageModel

C:\Users\Sian\AppData\Local\Temp\ipykernel_16856\2855830967.py:4: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


ModuleNotFoundError: No module named 'triton'

In [ ]:


# Define model and tokenizer
model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load base model with Unsloth
model, _ = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=512,  # Reduce sequence length for lower VRAM usage
    dtype=torch.float16,
    load_in_4bit=True,  # Optimize for low VRAM
)

# Load dataset
dataset = load_dataset("json", data_files="combined_dataset.jsonl")

# Tokenization function
def tokenize_function(example):
    return tokenizer(
        f"Question: {example['question']}\nA) {example['A']}\nB) {example['B']}\nC) {example['C']}\nD) {example['D']}\nAnswer: {example['answer']}",
        truncation=True,
        padding="max_length",
        max_length=512,
    )

# Tokenize dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Fine-tune model
trainer = FastLanguageModel.get_trainer(
    model,
    train_dataset=tokenized_dataset,
    batch_size=1,  # Small batch size to fit in 4GB VRAM
    gradient_accumulation_steps=8,  # Simulate a larger batch size
    learning_rate=1e-5,  # Adjusted for stability
    num_epochs=3,
)



In [ ]:
trainer.train()


In [ ]:

# Save fine-tuned model
model.save_pretrained("fine_tuned_phi2")
tokenizer.save_pretrained("fine_tuned_phi2")
